In [ ]:
import requests
import json
from mobfot import MobFot
from mplsoccer import Pitch

from PIL import Image
import urllib
import os

import pandas as pd
from bs4 import BeautifulSoup
# import klib as kb
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as path_effects
import matplotlib.font_manager as fm
# import wes
import matplotlib as mpl
import warnings
import numpy as np
from math import pi
from urllib.request import urlopen
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
from sklearn.preprocessing import StandardScaler


In [ ]:
font_path = "../assets/fonts"
for x in os.listdir(font_path):
    for y in os.listdir(f"{font_path}/{x}"):
        if y.split(".")[-1] == "ttf":
            fm.fontManager.addfont(f"{font_path}/{x}/{y}")
            try:
                fm.FontProperties(weight=y.split("-")[-1].split(".")[0].lower(), fname=y.split("-")[0])
            except Exception:
                continue

plt.style.use("../assets/stylesheets/soc_base.mplstyle")
plt.rcParams['font.family'] = 'Karla'

In [ ]:
team_table = pd.read_csv("CSVs/EPL_Teams_URLs.csv")
team_table = team_table[["team_names","urls"]]

def get_wages(url):    
    start = url[0:37]+ "2023-2024/wages/"
    def remove_first_n_char(org_str, n):
        mod_string = ""
        for i in range(n, len(org_str)):
            mod_string = mod_string + org_str[i]
        return mod_string
    mod_string = remove_first_n_char(url, 37)
    final_string = start+mod_string+"-Wage-Details"   
    return final_string

team_table['wages'] = team_table.apply(lambda x: get_wages(x['urls']), axis=1)

team_table

In [ ]:
import time
def league_wages_df(match_links):
    data_append = []
    for x in match_links:
        print(x)
        warnings.filterwarnings("ignore")
        url = x
        page =requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        name = [element.text for element in soup.find_all("span")]
        name = name[7]
        # name = name[10:]

        # Remove last 17 characters
        name = name[:-17]
        html_content = requests.get(url).text.replace('<!--', '').replace('-->', '')
        df = pd.read_html(html_content)
        wage_stats = df[0]
        def remove_nan_rows(df, column_name):
            df.dropna(subset=[column_name], inplace=True)
            return df
        wage_stats = remove_nan_rows(wage_stats, "Weekly Wages")
        def extract_currency_values(column_value):
            parts = column_value.split(" ")
            pound_value = parts[0] + "  " + parts[1]
            euro_value = parts[3]
            dollar_value = parts[4]
            return pound_value, euro_value, dollar_value

        wage_stats[["Pound Value", "Euro Value", "Dollar Value"]] = wage_stats["Weekly Wages"].apply(extract_currency_values).apply(pd.Series)
        def convert_pound_values_to_int(df, column_name):
            df['new_pound_value'] = df[column_name].str.replace('£', '').str.replace(',', '').astype(int)
            return df
        wage_stats = convert_pound_values_to_int(wage_stats, "Pound Value")
        wage_stats = wage_stats[["Player", "Nation","Pos","Age","new_pound_value"]]
        # wage_stats['Player'] = name
        data_append.append(wage_stats)
        del df, soup
        time.sleep(10)
    df_total = pd.concat(data_append)

    return df_total

In [ ]:
match_links = list(team_table.wages.unique())
wages_df = league_wages_df(match_links)
wages_df.to_csv("CSVs/EPL_Player_Wages.csv")

In [ ]:
import re
team_ids = pd.read_csv("CSVs/fotmob_epl_team_ids.csv")
client = MobFot()

def get_epl_ratings(team_ids):
    dataframes = []
    for t_id in team_ids.team_id.unique():
        dict1 = client.get_team(t_id)

        def extract_names_and_urls(data):
            names = []
            urls = []
            for athlete in data['details']['sportsTeamJSONLD']['athlete']:
                names.append(athlete['name'])
                urls.append(athlete['url'])
            df = pd.DataFrame({'Name': names, 'URL': urls})
            return df

        player_rating = extract_names_and_urls(dict1)

        fm_ratings = []
        player_ids = (player_rating.URL.unique())
        def extract_rating(player_ids):
            for id in player_ids:
                print(id)
                numbers = re.findall(r'\d+', id)
                id = ''.join(numbers)
                dict2 = client.get_player(id)
                l1= dict2['careerStatistics']
                def get_fotmob_rating(data, seasonid):
                    if data == None:
                        fm_ratings.append(None)
                    else:
                        for league in data:
                            for season in league['seasons']:
                                if season['seasonId'] == seasonid:
                                    for stat in season['stats'][0]['statsArr']:
                                        if stat[0] == 'FotMob rating':
                                            return stat[1]['value']['num']
                    return None
                rating = get_fotmob_rating(l1, 879290)  
                fm_ratings.append(rating)  # Append None if rating is not found

            return(fm_ratings)

        table = extract_names_and_urls(dict1)
        fm_ratings = extract_rating(player_ids)
        fm_ratings = fm_ratings[:len(table)]

        # Create a DataFrame with the player_ids and fm_ratings
        df2 = pd.DataFrame({'URL': player_ids, 'ratings': fm_ratings})

        # Merge the DataFrame with the original data using player_id as the key
        new_table = table.merge(df2, on='URL', how='left')

        new_table['team_id'] = t_id
        dataframes.append(new_table)
    final = pd.concat(dataframes)

    return final


In [ ]:
epl_player_ratings = get_epl_ratings(team_ids)

In [ ]:
wages_df = pd.read_csv("CSVs/EPL_Player_Wages.csv")
wages_df.rename(columns={'Player': 'Name'}, inplace=True)

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def get_approximate_match(query, choices):
    return process.extractOne(query, choices)[0]

# Create a dictionary to store approximate matches
name_mapping = {}

# Loop through each name in wages dataframe
for name in wages_df['Name']:
    # Find approximate match in epl_player_ratings dataframe
    match = get_approximate_match(name, epl_player_ratings['Name'])
    # Store the approximate match in the dictionary
    name_mapping[name] = match

# Replace the 'Name' column in wages dataframe with the approximate match
wages_df['Name'] = wages_df['Name'].map(name_mapping)

# Perform left join on 'Name' column
merged_df = pd.merge(wages_df, epl_player_ratings, on='Name', how='left')

# merged_df.drop('Unnamed: 0_y', axis=1, inplace=True)
# merged_df.drop('Unnamed: 0_x', axis=1, inplace=True)

# Remove rows with NaN values in 'Nation' column
merged_df.dropna(subset=['Nation'], inplace=True)
